In [66]:
%load_ext autoreload
%autoreload 2
    
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.linear_model import Ridge
from lineartree import LinearTreeRegressor
from lib_FM import *
import random
import time
import pickle
from warnings import simplefilter

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
random.seed(10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### READING THE DATASET

In [67]:
df=pd.read_csv('./fairlymade_products_impacts.csv')
df.columns

Index(['study_id', 'product_type', 'nb_components', 'composition',
       'raw_material_country', 'weaving_country', 'dyeing_country',
       'manufacturing_country', 'plane_in_transports', 'climate_change',
       'resource_use_fossils'],
      dtype='object')

### ENCODING CATEGORICAL VARIABLES AND BUILDING TRAIN/VALIDATION DATASETS

In [68]:
categorical_features=['product_type','nb_components']
multi_valued_features=['composition','raw_material_country','weaving_country','dyeing_country','manufacturing_country']
boolean_features=['plane_in_transports']
numerical_features=['resource_use_fossils']

FED=FeaturesEncoderDecoder(categorical_features=categorical_features,
                           multi_valued_features=multi_valued_features,
                           boolean_features=boolean_features,
                           numerical_features=numerical_features)

In [69]:
target_feature="climate_change"
df=df.dropna(subset=[target_feature]+numerical_features)
y=np.array(df[target_feature])

In [70]:
df=df.drop_duplicates()
t1=time.time()
X=FED.encode_dataframe(df,compute_labels=True)
t2=time.time()
print("encoding dataset took %f"%(t2-t1))


encoding dataset took 0.785401


In [71]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)


### TRAINING LINEAR MODEL TREES

In [72]:
best_score=None
alpha=1.
min_samples_leaf=3
min_impurity_decrease=0.00
for alpha in [0.01,1.,100.]:
    for min_samples_leaf in [0.0001,0.001,0.1]:
        for min_impurity_decrease in [0.0001,0.001,0.1]:
            reg=LinearTreeRegressor(base_estimator=Ridge(alpha=alpha),
                                    min_samples_leaf=min_samples_leaf,
                                    min_impurity_decrease=min_impurity_decrease,
                                    categorical_features=np.arange(FED.n_binary_features))
            scores=cross_val_score(reg,X_train,y_train)
            score=np.mean(scores)
            if best_score is None or best_score<score:
                best_score=score
                best_reg=reg
            print("alpha %f, min_samples_leaf %f, min_impurity_decrease %f : cv %s mean %f"%(alpha,min_samples_leaf,min_impurity_decrease,str(scores),score))
reg=best_reg

alpha 0.010000, min_samples_leaf 0.000100, min_impurity_decrease 0.000100 : cv [0.85449683 0.82296196 0.86250639 0.87011292 0.88898288] mean 0.859812
alpha 0.010000, min_samples_leaf 0.000100, min_impurity_decrease 0.001000 : cv [0.85449683 0.82296196 0.86250639 0.87011292 0.88898288] mean 0.859812
alpha 0.010000, min_samples_leaf 0.000100, min_impurity_decrease 0.100000 : cv [0.85449683 0.82296196 0.86250639 0.87011292 0.88898288] mean 0.859812
alpha 0.010000, min_samples_leaf 0.001000, min_impurity_decrease 0.000100 : cv [0.84870643 0.83085213 0.85797854 0.85221498 0.8904458 ] mean 0.856040
alpha 0.010000, min_samples_leaf 0.001000, min_impurity_decrease 0.001000 : cv [0.84870643 0.83085213 0.85797854 0.85221498 0.8904458 ] mean 0.856040
alpha 0.010000, min_samples_leaf 0.001000, min_impurity_decrease 0.100000 : cv [0.84870643 0.83085213 0.85797854 0.85221498 0.8904458 ] mean 0.856040
alpha 0.010000, min_samples_leaf 0.100000, min_impurity_decrease 0.000100 : cv [0.74935353 0.8040879

In [73]:
reg.fit(X_train,y_train)
reg.score(X_val,y_val),reg.score(X_train,y_train)

(0.90809322403741, 0.9025054576149235)